In [253]:
from random import *
class Maze:
    """
    Classe Labyrinthe
    Représentation sous forme de graphe non-orienté
    dont chaque sommet est une cellule (un tuple (l,c))
    et dont la structure est représentée par un dictionnaire
      - clés : sommets
      - valeurs : ensemble des sommets voisins accessibles
    """
    def __init__(self, height, width, empty):
        """
        Constructeur d'un labyrinthe de height cellules de haut
        et de width cellules de large
        Les voisinages sont initialisés à des ensembles vides
        Remarque : dans le labyrinthe créé, chaque cellule est complètement emmurée
        """
        self.height    = height
        self.width     = width
        self.empty     = empty
        self.neighbors = {(i,j): set() for i in range(height) for j in range (width)}

        if empty == True :
            self.neighbors = {(x,y): {(x-1,y),(x,y-1),(x+1,y),(x,y+1)} for x in range(height) for y in range (width)}
            for x in range(height):
                self.neighbors[x,width-1].remove((x,width))



    def info(self):
        """
        **NE PAS MODIFIER CETTE MÉTHODE**
        Affichage des attributs d'un objet 'Maze' (fonction utile pour deboguer)
        Retour:
            chaîne (string): description textuelle des attributs de l'objet
        """
        txt = "**Informations sur le labyrinthe**\n"
        txt += f"- Dimensions de la grille : {self.height} x {self.width}\n"
        txt += "- Voisinages :\n"
        txt += str(self.neighbors)+"\n"
        valid = True
        for c1 in {(i, j) for i in range(self.height) for j in range(self.width)}:
            for c2 in self.neighbors[c1]:
                if c1 not in self.neighbors[c2]:
                    valid = False
                    break
            else:
                continue
            break
        txt += "- Structure cohérente\n" if valid else f"- Structure incohérente : {c1} X {c2}\n"
        return txt

    def __str__(self):
        """
        **NE PAS MODIFIER CETTE MÉTHODE**
        Représentation textuelle d'un objet Maze (en utilisant des caractères ascii)
        Retour:
             chaîne (str) : chaîne de caractères représentant le labyrinthe
        """
        txt = ""
        # Première ligne
        txt += "┏"
        for j in range(self.width-1):
            txt += "━━━┳"
        txt += "━━━┓\n"
        txt += "┃"
        for j in range(self.width-1):
            txt += "   ┃" if (0,j+1) not in self.neighbors[(0,j)] else "    "
        txt += "   ┃\n"
        # Lignes normales
        for i in range(self.height-1):
            txt += "┣"
            for j in range(self.width-1):
                txt += "━━━╋" if (i+1,j) not in self.neighbors[(i,j)] else "   ╋"
            txt += "━━━┫\n" if (i+1,self.width-1) not in self.neighbors[(i,self.width-1)] else "   ┫\n"
            txt += "┃"
            for j in range(self.width):
                txt += "   ┃" if (i+1,j+1) not in self.neighbors[(i+1,j)] else "    "
            txt += "\n"
        # Bas du tableau
        txt += "┗"
        for i in range(self.width-1):
            txt += "━━━┻"
        txt += "━━━┛\n"

        return txt


    def add_wall(self, c1, c2):
        # Facultatif : on teste si les sommets sont bien dans le labyrinthe
        assert 0 <= c1[0] < self.height and \
            0 <= c1[1] < self.width and \
            0 <= c2[0] < self.height and \
            0 <= c2[1] < self.width, \
            f"Erreur lors de l'ajout d'un mur entre {c1} et {c2} : les coordonnées de sont pas compatibles avec les dimensions du labyrinthe"
        # Ajout du mur
        if c2 in self.neighbors[c1]:      # Si c2 est dans les voisines de c1
            self.neighbors[c1].remove(c2) # on le retire
        if c1 in self.neighbors[c2]:      # Si c3 est dans les voisines de c2
            self.neighbors[c2].remove(c1) # on le retire


    def remove_wall(self,c1,c2):
        # Facultatif : on teste si les sommets sont bien dans le labyrinthe
        assert 0 <= c1[0] < self.height and \
            0 <= c1[1] < self.width and \
            0 <= c2[0] < self.height and \
            0 <= c2[1] < self.width, \
            f"Erreur lors de l'ajout d'un mur entre {c1} et {c2} : les coordonnées de sont pas compatibles avec les dimensions du labyrinthe"
        # retirer le mur
        if c2 not in self.neighbors[c1]:
            self.neighbors[c1].add(c2)
        if c1 not in self.neighbors[c2]:
            self.neighbors[c2].add(c1)


    def get_walls(self):
        liste_mur=[]
        mur=[]
        for j in range(self.height):
            for i in range(self.width):
                if (j+1,i) not in self.neighbors[j,i] and (j+1<self.height):
                    mur = [(j,i),(j+1,i)]
                    if mur not in liste_mur:
                        liste_mur.append(mur)
                if (j,i+1) not in self.neighbors[j,i] and (i+1<self.width):
                    mur = [(j,i),(j,i+1)]
                    if mur not in liste_mur:
                        liste_mur.append(mur)
        return liste_mur



    def fill(self):
        """
        Ajoute tous les murs possibles dans le labyrinthe
        """
        for i in range(laby.height):
            for j in range(laby.width):
                c1=(i,j)
                for c2 in self.get_contiguous_cells(c1):
                    self.add_wall(c1, c2)


    def get_contiguous_cells(self, c):
        """
        Retourne la liste des cellules contigües à c dans la grille (sans s’occuper des éventuels murs)
        """
        i, j = c
        voisins = []
        if i > 0:
            voisins.append((i-1, j))
        if i < self.height-1:
            voisins.append((i+1, j))
        if j > 0:
            voisins.append((i, j-1))
        if j < self.width-1:
            voisins.append((i, j+1))
        return voisins


    def get_reachable_cells(self, c):
        """
        Retourne la liste des cellules accessibles depuis c (c’est-à-dire les cellules contiguës à c qui sont dans le voisinage de c)
        """
        cell_accessible = []
        for voisin in self.neighbors[c]:
            if voisin not in self.get_walls():
                cell_accessible.append(voisin)
        return cell_accessible


    def emptys(self):
        """
        Supprime tous les murs du labyrinthe
        """
        for i in range(laby.height):
            for j in range(laby.width):
                c1=(i,j)
                for c2 in self.get_contiguous_cells(c1):
                    self.remove_wall(c1, c2)

    def get_cells(self):
        """
        Retourne la liste de toutes les cellules de la grille du labyrinthe.
        """
        L = []
        for i in range(laby.height):
            for j in range(laby.width):
                L.append((i,j))
        return L

    def gen_btree(height,width):
        laby = Maze(height, width, False) # un labyrinthe plein (contenant tous les murs possibles)
        for i in range(laby.height):
            for j in range(laby.width):
                c1 = (i,j)
                voisins_EST = None
                if j<width-1:
                    voisins_EST = (i,j+1) # voisins EST
                voisins_SUD = None
                if i<height-1:
                    voisins_SUD = (i+1,j) # voisins SUD
                
                if voisins_SUD != None and voisins_EST != None:
                    if not voisins_EST in laby.get_reachable_cells(c1) and not voisins_SUD in laby.get_reachable_cells(c1): # si le mur EST et SUD existe
                        aleatoire = randint(1,2)
                        if aleatoire == 1: # supprimer le mur EST
                            laby.remove_wall(c1,voisins_EST)
                        else: # supprimer le mur SUD
                            laby.remove_wall(c1,voisins_SUD)
                            
                elif voisins_EST != None and not voisins_EST in laby.get_reachable_cells(c1): # si le mur EST existe mais pas le mur SUD, supprimer le mur EST
                    laby.remove_wall(c1,voisins_EST)

                elif voisins_SUD != None and not voisins_SUD in laby.get_reachable_cells(c1): # si le mur SUD existe mais pas le mur EST, supprimer le mur SUD
                    laby.remove_wall(c1,voisins_SUD)

        return laby
    
    def gen_sidewinder(height,width):
        laby = Maze(height, width, False) # un labyrinthe plein (contenant tous les murs possibles)
        for i in range(height-1):
            sequence = []
            for j in range(width):
                cell = (i,j)
                voisins_EST = None
                if j<width-1:
                    voisins_EST = (i,j+1) # voisins EST
                voisins_SUD = None
                if i<height-1:
                    voisins_SUD = (i+1,j) # voisins SUD
                sequence.append(cell)
                
                aleatoire = randint(1,2)
                if aleatoire == 1 and voisins_EST != None:
                    laby.remove_wall(cell,voisins_EST)
                elif aleatoire == 2 and voisins_SUD != None:
                    cell_random = sequence[randint(0,len(sequence)-1)]
                    x,y = cell_random
                    laby.remove_wall(cell_random,(x+1,y))
                    sequence = []
            sequence.append(cell)
            cell_random = sequence[randint(0,len(sequence)-1)]
            x,y = cell_random
            laby.remove_wall(cell_random,(x+1,y))
        for i in range(width-1):
            laby.remove_wall((height-1,i),(height-1,i+1))
        return laby
                    
    def gen_fusion(height,width):
        
        

In [254]:
laby = Maze.gen_btree(4, 4)
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃       ┃   ┃   ┃
┣━━━╋   ╋   ╋   ┫
┃       ┃       ┃
┣━━━╋   ╋━━━╋   ┫
┃       ┃   ┃   ┃
┣━━━╋   ╋   ╋   ┫
┃               ┃
┗━━━┻━━━┻━━━┻━━━┛



In [349]:
laby = Maze.gen_sidewinder(4, 4)
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃
┣   ╋   ╋   ╋   ┫
┃   ┃           ┃
┣   ╋━━━╋━━━╋   ┫
┃   ┃   ┃       ┃
┣   ╋   ╋   ╋   ┫
┃               ┃
┗━━━┻━━━┻━━━┻━━━┛

